In [ ]:
# generic
import pandas as pd
import os, glob
import numpy as np
import scipy as sp

# geospatial plotting
import cartopy
import cartopy.crs as ccrs 
import cartopy.feature as cfeature
from cartopy import config

# display
from IPython.display import Image
from cartopy.io.img_tiles import GoogleTiles

# plotting
import matplotlib.pyplot as plt
import seaborn as sns

# handling netCDFs
import xclim as xc
import xarray as xr
from netCDF4 import Dataset as netcdf_dataset

In [ ]:
# open a netCDFs downloaded from the curl script
# read in all the netcdfs in a folder, and make a dict with the run number as the key and xarrays values

def parse_data(fp_to_nc_folder):

    # lists to hold the netcdfs and their run numbers 
    run_numbers = []
    nc_names = []

    # read all the files matching the input file path
    for nc_file in glob.glob(fp_to_nc_folder):

        # open them!
        with open(os.path.join(os.getcwd(), nc_file), 'r') as f: 

            # extract the run number from the file name and add it to a list to ID each file
            # this assumes all ARISE-SAI files have more or less the same filename and location of the run number within it, may need to be updated
            run_numbers.append(pd.Series(nc_file).str.split(pat = 'DEFAULT.')[0][1].split('.cam')[0])

            # open each nc file and add that to another list
            nc_names.append(xr.open_dataset(nc_file))

    # combine the lists into a dict ordered by keys (run numbers low to high)
    data_dict = dict(sorted(dict(zip(run_numbers, nc_names)).items()))

    return(data_dict)

# use this function on the TMSO2 data
TMSO2_dict = parse_data('./project-data/TMSO2/*.nc')

In [ ]:
# check array dimensions
TMSO2_dict['001'].variables['TMSO2'][0,:,:].shape, dataset.variables['lat'].shape, dataset.variables['lon'].shape

In [ ]:
dataset = TMSO2_dict['001'].sel(time=slice("2035-01-01", "2036-01-02"))

tmso2 = dataset.variables['TMSO2'][0, :, :]
lats = dataset.variables['lat'][:]
lons = dataset.variables['lon'][:]

In [ ]:
# make a function that plots the variable of interest around the globe using cartopy
ax = plt.axes(projection=ccrs.PlateCarree())
plt.contourf(lons, lats, tmso2, transform=ccrs.PlateCarree())
# ax.coastlines()


In [ ]:
# The first five members of ARISE-SAI-1.5 simulations were initialized in 2035 from the first five members (001 to 005) 
# of the SSP2-45 simulations carried out with CESM2(WACCM6); hence, all had different initial ocean, sea-ice, land, 
# and atmospheric initial conditions on January 1, 2035. Similarly to the SSP2-45 simulations, subsequent ensemble members 
# (006 through 010) were initialized from the same initial conditions as members 001 through 005, respectively, 
# with an addition of a small temperature perturbation to the atmospheric initial condition to create ensemble spread.
# ([6, 7, 8, 9, 10] x 10ˆ-14 K, respectively)